In [2]:
import json
import re

import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta


def API_INGURU(url, per_page, start_date, end_date):
    headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36',
    'accept':'application/json',
    'authorization': '6f64d6a047fad369d35c3806f8f8e0560475075a432585973f91caae35b4ad74'
    } 

    params = {
        'per_page': per_page,
        'start_date': start_date,
        'end_date': end_date,
        'sort': '1'
    }
    response = requests.get(url, headers=headers, params=params)
    return response.json()

def divide_data(data_inicial, data_final):
    formato_data = '%d/%m/%Y %H:%M'

    data_inicial = datetime.strptime(data_inicial, formato_data)
    data_final = datetime.strptime(data_final, formato_data)
    intervalo = data_final - data_inicial
    data_final = data_inicial + intervalo // 2
    data_final = data_final.replace(hour=23, minute=59, second=59) # define o final como o último segundo do minuto
    data_final = data_final.strftime(formato_data)
    data_inicial = data_inicial.strftime(formato_data)
    return (data_inicial, data_final)


def get_news(data_inicial, data_final, id):
    url = 'https://app.inguru.me/api/v1/taxonomies/nodes/news/'+str(id)

    array_dfs = []

    getted_news = 0
    
    total_news = API_INGURU(url, 1, data_inicial, data_final)['pagination']['total']

    
    if not total_news:
        parcial_data_inicial = data_inicial
        parcial_data_final = data_final
        dados = {
        'author': np.nan,
        'content': np.nan,
        'crawled_date': np.nan,
        'domain': np.nan,
        'id': np.nan,
        'published_date': np.nan,
        'source': np.nan,
        'source_country': np.nan,
        'source_state': np.nan,
        'subtitle': np.nan,
        'title': np.nan,
        'url': np.nan,
        'dh_insertion_raw': np.nan
        }
        fail_df = pd.DataFrame(dados, index=[0])
        array_dfs.append(fail_df)
        parcial_data_inicial = parcial_data_final
        parcial_data_final = data_final
    else:
        
        print('Total de notícias: ', total_news)

        parcial_data_inicial = data_inicial
        parcial_data_final = data_final

        while getted_news < total_news:

            parcial_news = total_news

            while parcial_news > 10000:
                print('Parcial de notícias: ', parcial_news)
                parcial_data_inicial, parcial_data_final = divide_data(parcial_data_inicial, parcial_data_final)
                parcial_news = API_INGURU(url, 1, parcial_data_inicial, parcial_data_final)['pagination']['total']

            print("fazendo requisicao")
            print('Data inicial: ', parcial_data_inicial)
            print('Data final: ', parcial_data_final)
            news = API_INGURU(url, parcial_news, parcial_data_inicial, parcial_data_final)['data']

            #CRIA UM DATAFRAME COM AS NOTICIAS
            noticias = pd.DataFrame(news)
            #ADICIONA NUM ARRAY DE DATAFRAME
            array_dfs.append(noticias)

            #corrige as datas

            parcial_data_inicial = parcial_data_final
            parcial_data_final = data_final

            getted_news += parcial_news
    return pd.concat(array_dfs)


def insere_news(data_inicial, data_final, id, path):
    url = 'https://app.inguru.me/api/v1/taxonomies/nodes/news/'+str(id)

    array_dfs = []

    getted_news = 0

    total_news = API_INGURU(url, 1, data_inicial, data_final)['pagination']['total']

    if total_news:
        
    #     print('Total de notícias: ', total_news)

        parcial_data_inicial = data_inicial
        parcial_data_final = data_final

        while getted_news < total_news:

            parcial_news = total_news

            while parcial_news > 10000:
    #         print('Parcial de notícias: ', parcial_news)
                parcial_data_inicial, parcial_data_final = divide_data(parcial_data_inicial, parcial_data_final)
                parcial_news = API_INGURU(url, 1, parcial_data_inicial, parcial_data_final)['pagination']['total']

    #       print("fazendo requisicao")
    #       print('Data inicial: ', parcial_data_inicial)
    #       print('Data final: ', parcial_data_final)
            news = API_INGURU(url, parcial_news, parcial_data_inicial, parcial_data_final)['data']

            #CRIA UM DATAFRAME COM AS NOTICIAS
            noticias = pd.DataFrame(news)
            display(noticias)
            #corrige as datas
            parcial_data_inicial = parcial_data_final
            parcial_data_final = data_final

            getted_news += parcial_news


            print("Total:", total_news)
            print("Inseridas:", getted_news)

    else:
        print("Sem dados")

if __name__ == "__main__":
    graph_ids = []
    graph_names = []
    graph_dfs = []

    nodeID = '926' #Declaração do ID

    url_nodeID = 'https://app.inguru.me/api/v1/taxonomies/nodes/'+str(nodeID)
    IDs = API_INGURU(url = url_nodeID, per_page=1, start_date='', end_date='')['data'][0]
    graph_ids.append(IDs['id'])
    graph_names.append(IDs['name'].replace(" ", "_"))

    #Passa por cada "filho" no nó principal, caso não tenha nenhum, nada acontece.
    for node_children in IDs['children']:
        graph_ids.append(node_children['id'])
        graph_names.append(node_children['name'].replace(" ", "_"))

    data_inicial = '01/01/2015 00:00'
    data_final = datetime.now().strftime("%d/%m/%Y %H:%M")

In [15]:
headers = {
'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36',
'accept':'application/json',
'authorization': '6f64d6a047fad369d35c3806f8f8e0560475075a432585973f91caae35b4ad74'
} 
nodeID = '926'
per_page=1
start_date=''
end_date=''

params = {
    'per_page': per_page,
    'start_date': start_date,
    'end_date': end_date,
    'sort': '1'
}

response = requests.get('https://app.inguru.me/api/v1/taxonomies/nodes/'+str(nodeID), headers=headers)
response.json()


{'data': [{'id': 8774,
   'created_at': '2022-05-03 13:44:51',
   'updated_at': '2023-02-22 22:56:51',
   'name': 'Nodo Raiz',
   'expression': '"ACIDENTE" OR "ACIDENTES"',
   'initial_date': None,
   'final_date': None,
   'color': '#80B62E',
   'children': [{'id': 8776,
     'created_at': '2022-05-03 13:47:55',
     'updated_at': '2023-02-19 20:40:17',
     'name': 'CASOS',
     'expression': '"se acidentou" OR "acidente ocorreu" OR "aconteceu" OR "registro" OR "registrado" OR "sofreu" OR "feriu" OR "machucou" AND ("TRABALHO" OR "NO TRABALHO") NOT ("morte" OR "obito" OR "faleceu" OR "faleceram" OR "morreu" OR "morreram")',
     'initial_date': None,
     'final_date': None,
     'color': '#80B62E',
     'children': [],
     'links': {'news': 'https://app.inguru.me/api/v1/taxonomies/nodes/news/8776'}},
    {'id': 8775,
     'created_at': '2022-05-03 13:45:23',
     'updated_at': '2023-02-22 22:56:51',
     'name': 'ACIDENTES DE TRABALHO',
     'expression': '"acidentes de trabalho" OR

In [13]:
headers = {
'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36',
'accept':'application/json',
'authorization': '6f64d6a047fad369d35c3806f8f8e0560475075a432585973f91caae35b4ad74'
} 
nodeID = '926'
per_page=1
start_date=''
end_date=''

params = {
    'per_page': per_page,
    'start_date': start_date,
    'end_date': end_date,
    'sort': '1'
}

response = requests.get('https://app.inguru.me/api/v1/taxonomies/nodes/'+str(nodeID), headers=headers, params=params)
response.json()

graph_ids = []
graph_names = []
graph_dfs = []

IDs = API_INGURU(url = url_nodeID, per_page=1, start_date='', end_date='')['data'][0]
graph_ids.append(IDs['id'])
graph_names.append(IDs['name'].replace(" ", "_"))

print('graph_ids',graph_ids)
print('graph_names',graph_names)

graph_ids [8774]
graph_names ['Nodo_Raiz']


In [14]:
for node_children in IDs['children']:
    graph_ids.append(node_children['id'])
    graph_names.append(node_children['name'].replace(" ", "_"))
print('graph_ids',graph_ids)
print('graph_names',graph_names)

graph_ids [8774, 8776, 8775]
graph_names ['Nodo_Raiz', 'CASOS', 'ACIDENTES_DE_TRABALHO']


In [10]:
print(response.json()['data'][0])

{'id': 8774, 'created_at': '2022-05-03 13:44:51', 'updated_at': '2023-02-22 22:56:51', 'name': 'Nodo Raiz', 'expression': '"ACIDENTE" OR "ACIDENTES"', 'initial_date': None, 'final_date': None, 'color': '#80B62E', 'children': [{'id': 8776, 'created_at': '2022-05-03 13:47:55', 'updated_at': '2023-02-19 20:40:17', 'name': 'CASOS', 'expression': '"se acidentou" OR "acidente ocorreu" OR "aconteceu" OR "registro" OR "registrado" OR "sofreu" OR "feriu" OR "machucou" AND ("TRABALHO" OR "NO TRABALHO") NOT ("morte" OR "obito" OR "faleceu" OR "faleceram" OR "morreu" OR "morreram")', 'initial_date': None, 'final_date': None, 'color': '#80B62E', 'children': [], 'links': {'news': 'https://app.inguru.me/api/v1/taxonomies/nodes/news/8776'}}, {'id': 8775, 'created_at': '2022-05-03 13:45:23', 'updated_at': '2023-02-22 22:56:51', 'name': 'ACIDENTES DE TRABALHO', 'expression': '"acidentes de trabalho" OR "acidente de trabalho" OR "acidente no trabalho" OR "acidentes no trabalho" NOT ("DIÁRIO OFICIAL" OR "

In [18]:
data_inicial = '01/01/2015 00:00'
print(data_inicial)
data_final = datetime.now().strftime("%d/%m/%Y %H:%M")
print(data_final)

01/01/2015 00:00
08/03/2023 14:43


In [25]:
raph_ids = []
graph_names = []
graph_dfs = []

nodeID = '927' #Declaração do ID

url_nodeID = 'https://app.inguru.me/api/v1/taxonomies/nodes/'+str(nodeID)
IDs = API_INGURU(url = url_nodeID, per_page=1, start_date='', end_date='')['data'][0]
graph_ids.append(IDs['id'])
graph_names.append(IDs['name'].replace(" ", "_"))

In [27]:
#Passa por cada "filho" no nó principal, caso não tenha nenhum, nada acontece.
for node_children in IDs['children']:
    graph_ids.append(node_children['id'])
    graph_names.append(node_children['name'].replace(" ", "_"))

In [28]:
print(graph_ids)
print(graph_names)

[8774, 8776, 8775, 8777, 8780, 8780]
['Nodo_Raiz', 'NODO_ACIDENTES_DE_TRABALHO_COM_ÓBITOS', 'NODO_ACIDENTES_DE_TRABALHO_COM_ÓBITOS']


In [29]:
end_date = datetime.now().strftime("%d/%m/%Y %H:%M")
print(end_date)

start_date = (datetime.now() - timedelta(days=7)).strftime("%d/%m/%Y %H:%M")
print(start_date)

08/03/2023 17:26
01/03/2023 17:26


In [30]:
nodeID = '929'

In [31]:

headers = {
'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36',
'accept':'application/json',
'authorization': '6f64d6a047fad369d35c3806f8f8e0560475075a432585973f91caae35b4ad74'
} 
 
params = {
    'per_page': per_page,
    'start_date': start_date,
    'end_date': end_date,
    'sort': '1'
    }

per_page=1
start_date=''
end_date=''

url = 'https://app.inguru.me/api/v1/taxonomies/nodes/news/'+str(id)

response = requests.get(url, headers=headers, params=params)


In [ ]:
url = 'https://app.inguru.me/api/v1/taxonomies/nodes/news/'+str(id)

array_dfs = []

getted_news = 0

total_news = API_INGURU(url, 1, data_inicial, data_final)['pagination']['total']

In [ ]:
def get_news(data_inicial, data_final, id):
    url = 'https://app.inguru.me/api/v1/taxonomies/nodes/news/'+str(id)

    array_dfs = []

    getted_news = 0
    
    total_news = API_INGURU(url, 1, data_inicial, data_final)['pagination']['total']

    
    if not total_news:
      parcial_data_inicial = data_inicial
      parcial_data_final = data_final
      dados = {
        'author': np.nan,
        'content': np.nan,
        'crawled_date': np.nan,
        'domain': np.nan,
        'id': np.nan,
        'published_date': np.nan,
        'source': np.nan,
        'source_country': np.nan,
        'source_state': np.nan,
        'subtitle': np.nan,
        'title': np.nan,
        'url': np.nan,
        'dh_insertion_raw': np.nan
      }
      fail_df = pd.DataFrame(dados, index=[0])
      array_dfs.append(fail_df)
      parcial_data_inicial = parcial_data_final
      parcial_data_final = data_final
    else:
      
      print('Total de notícias: ', total_news)

      parcial_data_inicial = data_inicial
      parcial_data_final = data_final

      while getted_news < total_news:

          parcial_news = total_news

          while parcial_news > 10000:
              print('Parcial de notícias: ', parcial_news)
              parcial_data_inicial, parcial_data_final = divide_data(parcial_data_inicial, parcial_data_final)
              parcial_news = API_INGURU(url, 1, parcial_data_inicial, parcial_data_final)['pagination']['total']

          print("fazendo requisicao")
          print('Data inicial: ', parcial_data_inicial)
          print('Data final: ', parcial_data_final)
          news = API_INGURU(url, parcial_news, parcial_data_inicial, parcial_data_final)['data']

          #CRIA UM DATAFRAME COM AS NOTICIAS
          noticias = pd.DataFrame(news)
          #ADICIONA NUM ARRAY DE DATAFRAME
          array_dfs.append(noticias)

          #corrige as datas

          parcial_data_inicial = parcial_data_final
          parcial_data_final = data_final

          getted_news += parcial_news
          

    return pd.concat(array_dfs)